In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import keras

from keras import metrics
from keras import callbacks
from keras import models

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout
from keras.layers import GRU, Input, concatenate, ConvLSTM2D, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

Using TensorFlow backend.


In [2]:
def create_rnn_model(X):
      
  # RNN layers
    gru_layer_1 = GRU(units=GRU_UNITS, 
                 dropout=0.1, recurrent_dropout=0.1, 
                 return_sequences=True,
                 input_shape=(X.shape[1], X.shape[2]))
  
    model.add(gru_layer_1)
    model.add(Activation('relu'))
  
    gru_layer_2 = GRU(units=GRU_UNITS_2, 
                 dropout=0.1, recurrent_dropout=0.1, 
                 return_sequences=True)

    model.add(gru_layer_2)
    model.add(Activation('relu'))
    
    gru_layer_3 = GRU(units=GRU_UNITS_3, 
                 dropout=0.1, recurrent_dropout=0.1, 
                 return_sequences=True)

    model.add(gru_layer_3)
    model.add(Activation('relu'))
    
    model.add(Dense(256, activation='relu'))
  
    model.add(gru_layer_2)
    model.add(Activation('relu'))
    
  # Output layers
    output = Dense(LABEL_COUNT, activation='softmax')
#   drpout = Dropout(0.08, seed=42)
    model.add(output)
#   model.add(drpout)
  
    return model

In [ ]:
def cnn_model(X):
    model = Sequential()
    # CNN
    model.add(Conv1D(filters,(2),input_shape=(X.shape[1],X.shape[2]), activation='relu'))
    model.add(MaxPooling1D(pool_size=(4)))
    model.add(Conv1D(filters,(2),input_shape=(X.shape[1],X.shape[2]), activation='relu'))
    model.add(MaxPooling1D(pool_size=(4)))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(LABEL_COUNT , activation='softmax'))
    return model

In [ ]:
def lstm_model(X):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(LABEL_COUNT, activation= 'softmax'))
    return model

In [2]:
def compile_model(rnn_model):
    optimizer = keras.optimizers.Adam(lr=initial_lrate)

    rnn_model.compile(optimizer=optimizer,
                  loss=keras.losses.categorical_crossentropy,
                  metrics=[metrics.categorical_accuracy])
    
    return rnn_model

In [5]:
drop = 0.9
epochs_drop = 9.0
initial_lrate = 0.001

BATCH_SIZE = 50
EPOCHS = 50

GRU_UNITS = 90
GRU_UNITS_2 = 90
GRU_UNITS_3 = 90

decay_rate = []

In [4]:
def scheduler(epoch):
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    decay_rate.append(lrate)
  
    return lrate

In [8]:
basepath = ''
chkpoint_path = basepath + 'model_chkpoints/weights.best.hdf5'

model_chkpoint = ModelCheckpoint(chkpoint_path, verbose=0, save_best_only=True, save_weights_only=True)
early_stop = EarlyStopping(monitor='categorical_accuracy', min_delta=0.1, patience=10, verbose=0)
lrate = LearningRateScheduler(scheduler, verbose=1)

In [3]:
def train_model(rnn_model, X_train, y_train, X_val, y_val):
    model_fit = rnn_model.fit(X_train, y_train,
                     batch_size=BATCH_SIZE,
                     epochs=EPOCHS,
                     validation_data=(X_val, y_val),
                     verbose=1,
                     callbacks=[model_chkpoint, early_stop, lrate])
    
    plt.plot(model_fit.history['categorical_accuracy'])
    plt.title('Categorical Acc. for GRU Units: ' + str(GRU_UNITS) + ', GRU Units (2): ' + str(GRU_UNITS_2) + ' ,and Batch_size: ' + str(BATCH_SIZE))

#     plt.savefig(basepath + 'Graphs/RNN_accuracy_' + str(GRU_UNITS) + '_' + str(GRU_UNITS_2) + '_' + str(BATCH_SIZE) + '.png', dpi=200)
    plt.show()
    
    return rnn_model

In [6]:
def get_y_predict(rnn_model, X_test):
    rnn_model.load_weights(chkpoint_path)
    y_pred = rnn_model.predict(X_test, batch_size=BATCH_SIZE)
    
    return y_pred

In [10]:
# get the max probability for label binarized y value
def convert_y(y):
  y_conv = [j for i in range(len(y)) for j in range(len(y[i][0])) if y[i][0][j]==np.amax(y[i][0])]
  return np.array(y_conv)

In [9]:
def get_label(y_test, y_pred):
    y_t = convert_y(y_test)
    y_p = convert_y(y_pred)